In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import tropycal.tracks as tracks


In [2]:

desired_basin = 'north_atlantic' # east_pacific or north_atlantic
year_start = 1998
year_end = 2024
dt1 = datetime(year=year_start,month=1,day=1)
dt2 = datetime(year=year_end,month=12,day=31)

In [3]:
# Generate a list of storm names and ATCF IDs

basin = tracks.TrackDataset(basin=desired_basin,source='hurdat',include_btk=False)


szn_list = []
for szn in range(year_start,year_end+1):
    one_season = basin.get_season(szn).to_dataframe().set_index(['id'])
    szn_list.append(one_season)
atcf_id_sname_list = pd.concat(szn_list,axis=0)['name']

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (3.24 seconds)


In [4]:
# Grab with regular SHIPS

# Variables from SHIPS. HSTA and HEND are variables I made up for hours after RI start and hours before RI end.
variables = ['LAT','LON','TYPE','VMAX','SHRD','SHTD','VMPI','COHC','RSST','RHLO','RHMD','IR00','DTL']

basin = 'north_atlantic' # east_pacific or north_atlantic

# # Read in RI cases
# RI = pd.read_csv('/Users/acheung/data/RI_Cases/RI_Events_'+basin+'.csv')

# # Convert columns to datetime format
# RI["RI Start"] = pd.to_datetime(RI["RI Start"])
# RI["RI End"] = pd.to_datetime(RI["RI End"])

list_pds = [] # Saves all the information as one big list!

for var_ind in range(len(variables)): # Loops through all variables

    SHIPS_data = pd.read_csv('/Users/acheung/data/SHIPS/SHIPS_'+variables[var_ind]+'_'+desired_basin+'.csv')
    # Generate storm names list from ATCF IDs
    SHIPS_data['Time'] = pd.to_datetime(SHIPS_data['Time'])
#     SHIPS_data = SHIPS_data.set_index('Time')
    SHIPS_data = SHIPS_data.set_index(['Time','Storm_ID'])
    SHIPS_trimmed = SHIPS_data.iloc[(SHIPS_data.index.get_level_values(0) >= dt1 ) &
                                    (SHIPS_data.index.get_level_values(0) <= dt2)]
    list_pds.append(SHIPS_trimmed)

# Concatenate all the pandas arrays
SHIPS_concat = pd.concat(list_pds,axis=1)

# # Save to CSV
# SHIPS_concat.to_csv("/Users/acheung/data/RI_Cases/RI_Events_with_SHIPS_Data_"+basin+".csv")


In [5]:
# Can open any file for generating list of storm names from SHIPS ATCF IDs
SHIPS_data = pd.read_csv('/Users/acheung/data/SHIPS/SHIPS_'+variables[0]+'_'+desired_basin+'.csv')

# Generate storm names list from ATCF IDs
names_list = []
for id_ind in range(len(SHIPS_concat.index.get_level_values(1))):
    id_now = SHIPS_concat.index.get_level_values(1)[id_ind]
    name_now = atcf_id_sname_list.loc[id_now]
    names_list.append(name_now)

In [6]:
names_pd = pd.DataFrame(names_list,index=SHIPS_concat.index,columns=['Name'])
SHIPS_with_name = pd.concat((SHIPS_concat,names_pd),axis=1)
SHIPS_with_name = SHIPS_with_name.reset_index().set_index(['Time','Storm_ID','Name'])
# # Save to CSV
SHIPS_with_name.to_csv("/Users/acheung/data/SHIPS/all_SHIPs_data_combined_"+desired_basin+".csv")

In [7]:
SHIPS_with_name

,,,LAT,LON,TYPE,VMAX,SHRD,SHTD,VMPI,COHC,RSST,RHLO,RHMD,IR00,DTL
Time,Storm_ID,Name,,,,,,,,,,,,,
1998-07-27 12:00:00,AL011998,ALEX,11.3,25.4,1,25,1.7,320,126,13,27.4,68,67,26,934
1998-07-27 18:00:00,AL011998,ALEX,11.7,27.2,1,25,4.4,31,127,14,27.4,69,67,26,1099
1998-07-28 00:00:00,AL011998,ALEX,12.2,29.2,1,25,5.9,5,130,16,27.4,71,68,22,1292
1998-07-28 06:00:00,AL011998,ALEX,12.6,31.3,1,25,5.6,34,125,17,27.2,71,69,30,1506
1998-07-28 12:00:00,AL011998,ALEX,12.9,33.3,1,30,6.1,66,119,13,27.1,71,70,30,1714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-28 18:00:00,AL202023,TAMMY,33.3,58.0,1,40,36.6,97,64,0,999.9,58,51,2,1312
2023-10-23 12:00:00,AL212023,TWENTY-ONE,11.4,82.6,1,25,3.2,318,169,43,999.9,78,74,14,117
2023-10-23 18:00:00,AL212023,TWENTY-ONE,11.5,83.2,1,25,4.3,274,164,40,999.9,79,73,30,51


In [8]:
# # Grab with regular EC-SHIPS


# basin_EC = 'north_atlantic'

# RI_EC_pre = pd.read_csv('/Users/acheung/data/RI_Cases/RI_Events_'+basin_EC+'.csv')
# if basin_EC == 'north_atlantic':
#     EC_SHIPS_shear = pd.read_csv('/Users/acheung/data/EC_SHIPS/EC-SHIPS_deep_layer_shear_atl.csv')
# elif basin_EC == 'east_pacific':
#     EC_SHIPS_shear = pd.read_csv('/Users/acheung/data/EC_SHIPS/EC-SHIPS_deep_layer_shear_epac.csv')

# RI_EC = RI_EC_pre.where(RI_EC_pre['Season'] >= 2016).dropna()
    
# all_EC_shears = []
# for i in range(len(RI_EC)):
#     current_storm_shear_EC = EC_SHIPS_shear.where(EC_SHIPS_shear['Storm_ID'] ==
#                                             RI_EC.iloc[[i]]['Storm_ID'].values[0]).dropna()
#     if len(current_storm_shear_EC.where(
#         current_storm_shear_EC['Time'] == RI_EC.iloc[[i]]['RI Start'].values[0]).dropna()['Deep_Layer_Shear']) == 0:
#         # Some data is missing from EC-SHIPS, so we record NaN in these cases
#         all_EC_shears.append(np.nan)
#     else: # When data from EC-SHIPS is not missing
#         current_shear_RI = current_storm_shear_EC.where(
#             current_storm_shear_EC['Time'] == RI_EC.iloc[[i]]['RI Start'].values[0]).dropna()['Deep_Layer_Shear'].values[0]
#         all_EC_shears.append(current_shear_RI)
    
# RI_EC.insert(6, "Deep-Layer Shear (kt)", all_EC_shears)
# RI_EC.to_csv("/Users/acheung/data/RI_Cases/RI_Events_with_EC_SHIPS_shear_"+basin_EC+".csv",index=False)
